데이터베이스 : 데이터를 **구조화**하여 저장하는 시스템


ORM : 객체 단위로 데이터베이스를 다룰 수 있게 도와준다. django에는 ORM이 내장되어있다.

### Model은 `models.py`에서 관리한다.

models.py는 우리가 startapp으로 만들어낸 homepage라는 앱을 만들 때 자동적으로 생성이 됐다.


`models.py`에 새로운 모델을 만들면 class 단위로 만들게 된다.  
```
class <모델 이름>(상속하는 대상 = models.Model):
    field1 = models.FieldType().... # Field 1 - 실제 데이터의 column이 되는 attribute들
    field3 = models.FieldType().... # Field 3 - 실제 데이터의 column이 되는 attribute들
    field2 = models.FieldType().... # Field 2 - 실제 데이터의 column이 되는 attribute들
    """
    문자열 : CharField
    숫자 : IntegerField, SmallIntergerField, ...
    논리형 : BooleanField
    시간/날짜 : DateTimeField
    ...

    """
    '''
    각 Field에는 인자로서 제약사항 혹은 옵션을 넣어줄 수 있다.
    - default : 이 행의 기본값 지정
    - null : null 허용 여부, 이 field가 비어있어도 되는지를 나타낸다. null=False 이면 반드시 값이 채워져야 한다.
    '''
```

이 클래스를 바탕으로 만들어지는 객체는 하나의 row, 행이 된다.  
이 행은 각각의 Field를 column으로 가지게 된다.


```python

from django.db import models #default

class Coffee(models.Model):
    name = models.CharField(default="", max_length=30) # CharField는 max_length 매개변수가 반드시 필요하다.
    price = models.IntegerField(default=0)
    is_ice = models.BooleanField(default=False)

```

---

homepage app 의 `admin.py`를 보자.  


admin은 장고가 기본적으로 제공하는 관리자 이름으로 superuser를 이용해 접근할 수 있었다.


### Django admin의 가장 큰 특징 : 어떤 모델이 있을 때 이 모델을 자연스럽게 관리해줄 수 있다.

---

## 모델과 admin의 연동

1. admin.py로 가자.
2. 이전에 만들어두었던 models.py의 Coffee 클래스를 admin.py에서 불러올 수 있게 import 하자.
3. models.py와 admin.py는 동일 경로에 존재하므로 `.models`로 접근하자. `from .models import Coffee`
4. 이제 admin에서도 Coffee 클래스를 사용할 수 있다.
5. `admin.site.register(<모델이름>)`를 적어주자. `admin.site.register(Coffee)` 이 한 줄로 관리자 페이지에서 모델을 관리할 수 있게 됐다.
6. 하지만 관리자 페이지에서 Coffee 모델을 누르면 no such table 오류가 발생한다.
7. 이는 한 가지 설정을 아직 하지 않아서 그렇다.
8. 데이터베이스 관점에서 데이터베이스에 변동사항이 생기면 settings에서 반영을 해줘야한다.
9. 장고에서는 데이터베이스를 관리할 때 어떤 모델을 `models.py`에 만드는 것뿐만 아니라 git commit과 유사한 migration 단위로 관리한다.
10. DB정보나 DB field를 수정하더라도 당장은 반영되지 않고 migration을 진행한 다음에야 DB가 반영이 되는 것을 확인할 수 있다.


migration을 CLI환경에서 진행해보자.
### `python manage.py migrate`
admin에 적용되는 group과 user라는 데이터베이스도 django project를 실행할 때 동시에 만들어지는데, 이 또한 클래스 단위로 만들어진다.  
즉 실제로 반영이 안 된 상태였기 때문에 superuser를 만들 때도 먼저 migrate를 진행해서 반영하고 모델을 연동했었다.  


그렇다면 우리가 새로 만든 Coffee라는 모델도 바로 migrate를 하면 되는가?  
아니다.  


git commit 을 하기 전에 git add 를 해야하듯이 migration을 먼저 만들어서,  
django 환경에게 우리가 새 모델을 만들었다는 것을 알려줘야 한다.  
`python manage.py makemigrations <모델이 담긴 앱 지정>`


즉 `python manage.py makemigrations homepage`를 해주자.


그 다음에는 이 migration을 적용하면 된다.


`python manage.py migrate`  이것은 만들어진 migration들을 실제로 DB에 반영하는 과정이다.





---

#### 커피 객체를 Object의 나열이 아닌 이름으로 구분하자.

다시 `models.py`에 되돌아오자.


Coffee 행이 있을 때 각 커피의 이름이 뜨게 하고 싶다.


이는 파이썬의 특수메서드인 `__str__`을 사용하면 해결할 수 있다.


In [ ]:
class Coffee(models.Model):
    def __str__(self): # 커피 객체를 출력하는 과정에서 어떤 문자열을 보여줄 지 결정하는 함수
        return self.name
    name = models.CharField(default="", max_length=30) # CharField는 max_length 매개변수가 반드시 필요하다.
    price = models.IntegerField(default=0)
    is_ice = models.BooleanField(default=False)

migrate를 하지 않고 models.py를 저장한 후에 관리자 페이지 새로고침만 해도 확인 가능하다.


---

## Model을 Template 상에서 보여주자

Model에서 Template으로 정보를 전달하기 위해서는 반드시 중간에 View를 거쳐야 한다.


따라서 view를 먼저 작성하도록 하자.

In [ ]:
from django.shortcuts import HttpResponse, render
from .models import Coffee

# Create your views here.
def index(request): 
    nums = [1,2,3,4,5]
    return render(request ,'index.html' , {"my_list":nums})

def coffee_view(request):
    coffee_all = Coffee.objects.all() # Coffee에 해당하는 DB가 있을텐데 그것의 모든 object(모든 행)를 가지고 와라.
    return render(request,'coffee.html',{"coffee_list":coffee_all})

이제 `coffee.html` 이라는 Template를 만들어주자.

coffee.html
```html

<!DOCTYPE html>
<html>
    <head>
        <title>Coffee List</title>
    </head>

    <body>
        <h1>My Coffee List</h1>
        <p>{{ coffee_list }}</p>
        
    </body>
</html>

```

이제 이 경로를 지정해줘야 하므로 urls.py로 가서 패턴 등록을 해준다.


## django는 기본 구성이 어떻게 짜여있고, 또 흘러가는지 잘 기억하고 그대로 따라야 한다.


`from homepage.views import coffee_view`를 추가해주고  


`path('coffee/', coffee_view)`을 등록한다.

{{ coffee_list }} 는 
# My Coffee List
<QuerySet [<Coffee: Americano>, <Coffee: Ice Americano>, <Coffee: Cafe Latte>]>


이런 쿼리 셋을 반환한다.


이 쿼리 셋을 Template단위에서 파싱해서 하나하나 접근해주도록 하자.


```
<!DOCTYPE html>
<html>
    <head>
        <title>Coffee List</title>
    </head>

    <body>
        <h1>My Coffee List</h1>
        {% for coffee in coffee_list %}
            <p> {{ coffee.name }}, {{ coffee.price }}</p>
        {% endfor %}
        
    </body>
</html>
```


그럼
# My Coffee List
Americano, 4100


Ice Americano, 4100


Cafe Latte, 4600


이런 결과를 냄을 확인할 수 있다.

---
## View, Model, Template의 기본적인 Logic 

1. 먼저 모델을 만든다.
2. 모델을 잘 만들었다면 View에서의 작업이 시작된다.
3. 우선 모델을 import하고 그 모델에 해당하는 값을 가지고 온다. (ex. 모델.objects.all() | all() 대신에 get() 이나 filter() 로 특정 조건에 해당하는 행만 가져올 수도 있다.)
4. 가져온 객체를 (view에서 )dictionary에 담아, HTML Template으로 넘겨준다.
5. 이 정보를 넘겨받은 Template은 template 변수와 template tag를 이용하여 데이터들을 동적으로 처리해주면 된다.

---

## Template상에서 정보 수정하기
HTTP method로 치면 'POST'에 해당하는 작업


이를 위해서는 form을 만들어야 한다. 


1. 만들어두었던 app 안에 새로운 file `forms.py`을 만들자. (= 자동으로 만들어지는 파일을 제외하고는 최초로 만든 파일)
2. `from django import forms`로 import를 해준다.
3. 이 form과 연동할 모델을 가지고 오자. `from .models import Coffee`
4. 이 form을 만드는 과정에 있어서 어떤 form을 만들지 적어준다. (클래스 생성)
   - Model에 관한 form을 만들 것이다 -> forms.ModelForm을 상속받는 form을 만들자
   > form을 작성하는 형태에는 제약이 있기 때문에 만드는 방식을 유념하자.
5. class 안에 Meta class를 만들어준다.   
   - `class Meta:`를 해주는 이유 : 이 form을 만들기 위해서 어떤 모델이 쓰여야하는지 이 class 안의 class에서 지정해주기 때문
   - Meta 안에는 attribute를 2개 지정해준다.
   - `model =` Coffee : 모델 명시
   - `fields = ` ('name', 'price', 'is_ice'): 어떤 field를 이 form에서 받을 것인지 적는다.




In [ ]:
from django.forms import fields
from .models import Coffee # Model 호출

class CoffeeForm(forms.ModelForm): # ModelForm을 상속받는 CoffeeForm 생성
    class Meta:
        model = Coffee
        fields = ('name', 'price', 'is_ice')

여기까지 작성해서 form을 만들었으면, view에서도 이 정보를 template으로 전달해야 한다.  
(Model의 정보를 template으로 전달했던 것과 마찬가지.)

 `views.py`에 가서 Form을 import 한다.


`from .forms import CoffeeForm`


Form에서도 CoffeeForm은 단순한 클래스에 불과하다.  
이걸 실제로 활용하기 위해서는 이 클래스를 바탕으로 한 새로운 객체를 만들 필요가 있다.


`form = CoffeeForm()`


이제 이 객체를 value로써 전달하면 된다.




In [ ]:
def coffee_view(request):
    # 정보
    coffee_all = Coffee.objects.all() # .get(), ,filter(), ...
    # form
    form = CoffeeForm() 
    return render(request,'coffee.html',{"coffee_list":coffee_all, "coffee_form":form}) # 정보와 form이 전달된다

이 Form을 View로부터 Template으로 전달했으니, Template에서 활용하는 방법을 배워보자.


html에서는 form 태그를 열고 그 안에 view로부터 받은 form 객체를 넣어준다. 

```
<form>
    {{ coffee_form }}
</form>
```


이를 정확한 form의 형태로 나타내기 위해서는 as paragraph를 작성해준다. `.as_p`를 사용하자.

```
<form>
    {{ coffee_form.as_p }}
</form>
```

그럼 평소 봤던 형태로 rendering이 된다.


하지만 우리는 form에 대한 틀만 만들었지, form정보를 server로 전송하는 버튼은 만들지 않았다.


html 옵션을 적절히 추가하여 버튼을 추가해주도록 하자.


```
<form method = "POST">
    {{ coffee_form.as_p }}
    <button type="submit">Save</button>
</form>
```


버튼이 생긴 것을 확인할 수 있지만, 작성 후 누르면 `CSRF verification failed`안내와 함께 403 에러가 날 것이다.  


form에는 기본적으로 보안 옵션이 들어가야만 하기 때문에
django의 CSRF를 구현하지 않으면 403 오류가 발생한다.


즉, 우리가 server의 입장에서 CSRF token을 form 안에 삽입해줘야 한다는 소리다.


form 안에 CSRF token을 삽입하는 것은 굉장히 간단하다.

1. form의 여는 태그와 닫는 태그 사이에 template tag `{% %}`를 달고 
2. 그 안에 csrf_token을 적어 넣어주기만 하면 된다.


```
<form method="POST"> {% csrf_token %}
    {{ coffee_form.as_p }}
    <button type="submit">Save</button>
</form>
```




---

페이지를 새로고침해보고 저장을 하면 이제는 더이상 오류가 나지 않는 것을 볼 수 있다.


하지만 우리는 View에서 POST요청이 들어왔을 때 어떤 로직을 해야하는지 적지 않았기 때문에 DB에는 반영이 되지 않을 것이다.


이제 View에서 POST 요청이 들어왔을 때 어떻게 Model에 정보를 넣어줄 수 있을지 적어보자.

## View에서 Request 종류에 따라 해야할 task를 다르게 설정해보자

- request가 post일 때 : POST를 바탕으로 Form을 완성하고, Form이 유효하면 저장


1. request가 POST일 때 -> `method`라는 `attribute`로 Check
2. class 인자로 request.POST 넣기 -> 완성된 form
3. is_valid() 함수로 -> 유효확인



In [ ]:
def coffee_view(request):
    coffee_all = Coffee.objects.all() # Coffee에 해당하는 DB가 있을텐데 그것의 모든 object(모든 행)를 가지고 와라. # .get(), ,filter(), ...
    # 만약 request가 POSt라면:
            # POST를 바탕으로 Form을 완성하고
            # Form이 유효하면 -> 저장!
    if request.method == "POST":
        form = CoffeeForm(request.POST) # 완성된 Form
        # CoffeeForm을 만들 때 그 안에 request.POSt를 넣어주자
        # 즉 html 파일 상에서 POST로 보내줬던 내용을 바탕으로 이 폼을 완성시킨 것을 form이라고 하자는 의미이다.
        if form.is_valid(): # 채워진 Form이 유효하다면:
            # is_valid : form 안에 있는 값들이 유효한지를 check하는 함수 
            form.save() # 이 Form 내용을 Model에 저장
            # form.save()가 가능한 이유는 form을 정의할 당시에 Meta class에 해당하는 모델을 지정해두었기 때문이다.
    form = CoffeeForm()
    return render(request,'coffee.html',{"coffee_list":coffee_all, "coffee_form":form}) # 정보와 form이 전달된다

이렇게 우리는 form을 이용함과 동시에 GET과 POST에 따라서 View가 해줘야 할 일을 분리함으로써

Template 상에서 새로운 데이터를 추가하는 로직을 구현할 수 있었다.

## 정리 

1. form은 기본적으로 양식이다. 어떤 데이터베이스를 입력받기 위한 양식에 해당하는 객체, class이다.
2. 그 class를 만들어줄 당시에 어떤 모델에 대한 양식을 만들 것인지, 그리고 어떤 필드에 대해 입력을 받을 것인지 Meta class로 지정해준다.
3. 이것을 View단에서 인스턴스화, 즉 객체를 새로 만들어 Template으로 전달하게 된다.
4. Template에서는 template 변수와 `.as_p` 등을 이용해서 표현할 수 있었다.
5. 이로 인해 admin을 거치지 않고도 데이터베이스에 접근해서 데이터를 추가할 수 있었다.